In [17]:
import easyocr
import fitz  # PyMuPDF
from PyPDF2 import PdfReader
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from groq import Groq
import os
from dotenv import load_dotenv


from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader


from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_ollama.llms import OllamaLLM
from typing import Optional

In [18]:

load_dotenv()

## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY']
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


# categorize in geredient

## Recommand recepies using Groq

In [19]:
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text



def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    metadata = [{"source": f"Document {i}"} for i, chunk in enumerate(text_chunks)]  # Add document links or IDs here
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings, metadatas=metadata)
    vector_store.save_local("../db/faiss_index")



In [20]:
pdf_docs = [ "../data/cook_books/2022CookingAroundtheWorldCookbook.pdf",
            "../data/cook_books/cookbook.pdf"]
raw_text = get_pdf_text(pdf_docs)
text_chunks = get_text_chunks(raw_text)
#get_vector_store(text_chunks)

   



In [22]:
embeddings = OpenAIEmbeddings()
vectors = FAISS.load_local("../db/faiss_index", embeddings, allow_dangerous_deserialization=True)

In [23]:
llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="llama-3.1-70b-versatile",
             temperature=0.0)




prompt = ChatPromptTemplate.from_template(
"""
Your job is to find a recipe from the provided context that use the given ingredients. 
Ensure that each recipe is described with the following fields:
- **Recipe 1: 
- **name**: The name of the recipe.
- **preparation_time**: The time required to prepare the recipe.
- **directions**: A list of instructions for preparing the recipe.
- **ingredients**: A list of ingredients required for the recipe.
- **calories**: The total number of calories in the recipe.
- **total fat (PDV)**: Percentage of daily value for total fat.
- **sugar (PDV)**: Percentage of daily value for sugar.
- **sodium (PDV)**: Percentage of daily value for sodium.
- **protein (PDV)**: Percentage of daily value for protein.
- **saturated fat (PDV)**: Percentage of daily value for saturated fat.
- **carbohydrates (PDV)**: Percentage of daily value for carbohydrates.

The recipes must be selected from the context provided below. If any ingredients are missing from the list, include them in the recipe details.

If you cannot find a recipe that meets the criteria, please state that you don’t know.

<contex>
{context}
</context>

Questions:
{input}
"""
)





In [25]:

user_message="Find a recipe that includes chicken breast as an ingredient and has less than 200 calories per serving."
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vectors.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
response=retrieval_chain.invoke({'input':user_message})

In [26]:
for key, value in response.items():
    print()
    print("*"*20)
    print(key)
   # print(value)


********************
input

********************
context

********************
answer


In [27]:
response["context"]

[Document(metadata={'source': 'Document 19'}, page_content='2 Tablespoons spread (try reduced fat \ncream cheese, peanut butter, or sunflower \nseed butter)\n⅓ cup bite-sized fruit (try strawberries, crushed and drained \npineapple, raisins, chopped apples, pears or peaches)\n1 Tablespoon chopped nuts or seeds, if desired\nDirections\n1. Split open the English muffin and toast the halves until \nlightly browned.\n2. Divide the spread and place on both halves.\n3. Arrange the fruit on top, and nuts or seeds, if desired. \nEnjoy right away.\nIngredients\n1 small banana\n½ cup oat , corn,  or granola cereal\n½ cup low-fat vanilla or strawberry yogurt\n½ teaspoon honey, if desired (skip for children under the \nage of one)\n½ cup canned pineapple tidbits or chunks\nDirections\n1. Peel the banana and split it lengthwise. Place the halves \nin two separate cereal bowls.\n2. Over each banana spoon yogurt, sprinkle cereal and \ndrizzle honey, if desired.\n3. Top with pineapple and serve.Variat

In [28]:
for answer in response["answer"].split("\n\n"):
    print(answer)

**Recipe 1:**
- **name**: Curried Chicken Salad
- **preparation_time**: 30 minutes
- **directions**: 
  1. Fill a large sauté pan about halfway with water. Bring to a boil over high heat, then reduce the heat to maintain a gentle simmer.
  2. Add the chicken and salt, then cover with a lid. Cook until the chicken is cooked through (internal temperature of 165°F), about 10 -12 minutes. Transfer the chicken to a plate and let cool slightly.  
  3. When cool enough to handle, cut the chicken into ¼ -to ½-inch pieces and place in the refrigerator to chill, about 25-30 minutes (or about 10 -15 minutes in the freezer) .
  4. Meanwhile, add the mayonnaise, Greek yogurt, onion, lime juice, curry powder, honey, ginger, and black pepper to a large bowl. Whisk to combine.
  5. Add the mango, grapes, and chicken. Toss to combine and coat with the dressing.
  6. Serve right away, or chill before serving. When ready to serve, scoop each portion over 1 cup of the arugula.
- **ingredients**: 
  - 1½ p

In [37]:
for doc in response["context"]:
    if "Curried Chicken Salad" in doc.page_content:
        #print(doc.metadata["source"])
        # print(f"Document: {doc.metadata['source']}, Chunk: {doc.metadata['chunk_id']}")
        print(f"{doc.metadata['source']}")
        print("Text:", doc.page_content)
        print("\n")

Document 2
Text: hot sauce or chili oil. Set in the refrigerator to chill.
2.In a separate small bowl, stir together the salt and pepper.
3.Pat the chicken dry with paper towels, then sprinkle evenly with the salt -pepper mixture.
4.Heat a tagine, a Dutch oven, or a large stockpot over medium -high heat.
5.Add the oil and heat until shimmering.
6.Place the chicken in the pan and cook until golden -brown 
but not cooked through, about 3 -5 minutes on each side. 
Transfer the chicken to a plate and set aside.
7.Reduce the heat to medium, then add the onion and cook until softened, about 5 -7 minutes. Add the garlic and 
allspice. Cook until fragrant, about 1 -2 minutes.
8.Add the beans, tomatoes with their juice, apricots, and cinnamon. Bring to a simmer over medium -high heat, then 
reduce the heat to maintain a gentle simmer. 
9.Return the browned chicken to the pan and cover with a lid, 
Cook until the chicken is cooked through (internal 
temperature of 175°F), about 33 -35 minutes.
1